In [1]:
import os
import numpy as np
import pandas as pd
from ast import literal_eval
import matplotlib.pyplot as plt
import tqdm
import pickle

import torch
from torch_geometric.datasets import Planetoid
from torch_geometric.data import DataLoader , Data
from torch_geometric.nn import GCNConv, MessagePassing
from torch_geometric.utils import add_self_loops, degree, to_dense_adj

## GCN Layer
*SEMI-SUPERVISED CLASSIFICATION WITH GRAPH CONVOLUTIONAL NETWORKS*
$$
\mathbf{x}^{\prime}_i = \mathbf{\Theta}^{\top} \sum_{j \in \mathcal{N}(i) \cup \{ i \}} \frac{e_{j,i}}{\sqrt{\hat{d}_j\hat{d}_i}}\mathbf{x}_j
$$

- Node dimension - $n$ 
- Output dimension - $m$
- $x_{i}$ - Feature vector of node $i$
- $\Theta$ - Learnable parameter $(n*m)$
- $\mathcal{N}(i)$ - Neighbouhood of node $i$
- $e_{j,i}$ - edge weight of edge $(j,i)$
- $d_i$ - degree of node $i$



In [ ]:
class GCNConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super().__init__(aggr='add')  # "Add" aggregation (Step 5).
        self.lin = Linear(in_channels, out_channels, bias=False)
        self.bias = Parameter(torch.Tensor(out_channels))

        self.reset_parameters()

    def reset_parameters(self):
        self.lin.reset_parameters()
        self.bias.data.zero_()

    def forward(self, x, edge_index):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]

        # Step 1: Add self-loops to the adjacency matrix.
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))

        # Step 2: Linearly transform node feature matrix.
        x = self.lin(x)

        # Step 3: Compute normalization.
        row, col = edge_index
        deg = degree(col, x.size(0), dtype=x.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

        # Step 4-5: Start propagating messages.
        out = self.propagate(edge_index, x=x, norm=norm)

        # Step 6: Apply a final bias vector.
        out += self.bias

        return out

    def message(self, x_j, norm):
        # x_j has shape [E, out_channels]

        # Step 4: Normalize node features.
        return norm.view(-1, 1) * x_j

In [21]:
edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long) # graph connectivity in COO format
x = torch.tensor([[-1, 12, 32, 1, 3], [0, 3, 42, 12, 4], [1, 31,4, 1, 4]], dtype=torch.float) # Node feature matrix (scalers in this case)
data = Data(x=x, edge_index=edge_index)

In [32]:
gcn_layer_1 = GCNConv(in_channels=5, out_channels=3, bias=True, improved=True, )
gcn_layer_2 = GCNConv(in_channels=3, out_channels=1, bias=True)

In [33]:
x = data.x
x = gcn_layer_1(x, data.edge_index)
x = gcn_layer_2(x, data.edge_index)

In [40]:
for params in gcn_layer_1.parameters():
    print(params.shape)

torch.Size([3])
torch.Size([3, 5])


In [28]:
for params in gcn_layer_2.parameters():
    print(params)

Parameter containing:
tensor([0.], requires_grad=True)
Parameter containing:
tensor([[-1.1387,  0.7134, -0.2539]], requires_grad=True)


In [2]:
data = Planetoid('data/Planetoid', name='cora')